In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws
import cloudknot as ck
import diff_classifier.knotlets as kn
import numpy as np

In [2]:
rows = ['0', '1', '2', '3']
folder = '01_18_Experiment'

missing = []
for i in range(10, 15):
    for row in rows:
        for col in rows:
            missing.append("{}/P1/P1_S2_R_00{}_{}_{}.tif".format(folder, i, row, col))

for i in range(10, 15):
    for row in rows:
        for col in rows:
            missing.append("{}/P2/P2_S3_L_00{}_{}_{}.tif".format(folder, i, row, col))
    
for i in range(0, 15):
    for row in rows:
        for col in rows:
            missing.append("{}/P3/P3_S3_L_{}_{}_{}.tif".format(folder, "%04d" % i, row, col))

In [ ]:
req = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'))
req_string = req.read()
req.close()

new_req = req_string[0:req_string.find('\n')-1]+'8'+ req_string[req_string.find('\n'):]
req_overwrite = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'), 'w')
req_overwrite.write(new_req)
req_overwrite.close()

In [ ]:
to_track = {}
knot = {}
result_futures = {}
start_knot = 50

pups = ["P1"]
slices = ["S1", "S2", "S3"]
folder = '01_18_Experiment'
start_vid = [0, 3, 6, 9, 12]
end_vid = [3, 6, 9, 12, 15]

hemis = ["R", "L"]
rows = ['0', '1', '2', '3']

github_installs=('https://github.com/ccurtis7/diff_classifier.git')
my_image = ck.DockerImage(func=kn.download_and_track, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

cut1 = docker_string.find('# Create a default user')
cut2 = docker_string[cut1:].find('\n\n') + cut1
new_docker = docker_string[0:cut1]+docker_string[cut2:]
docker_overwrite = open(my_image.docker_path, 'w')
docker_overwrite.write(new_docker)
docker_overwrite.close()

req = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'))
req_string = req.read()
req.close()

new_req = req_string[0:req_string.find('\n')-2]+'28'+ req_string[req_string.find('\n'):]
req_overwrite = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'), 'w')
req_overwrite.write(new_req)
req_overwrite.close()

for pup in pups:
    for slic in slices:
        for start_ind in range(0, 5):

            run_name = '{}_{}_{}'.format(pup, slic, start_ind)
            to_track[run_name] = []
            for hemi in hemis:
                for i in range(start_vid[start_ind], end_vid[start_ind]):
                    for row in rows:
                        for col in rows:
                            name = "{}/{}/{}_{}_{}_{}_{}_{}.tif".format(folder, pup, pup, slic, hemi, "%04d" % i, row, col)
                            if not name in missing:
                                to_track[run_name].append(name)

            test_length = len(to_track[run_name])
            print('Number of nodes to be loaded: {}'.format(test_length))
            
            knot[run_name] = ck.Knot(name='download_and_track_with_my_image_{}'.format(start_knot),
                           docker_image = my_image,
                           pars_policies=('AmazonS3FullAccess',))
            result_futures[run_name] = knot[run_name].map(to_track[run_name])
            start_knot = start_knot + 1
            print('Next knot name: {}'.format(start_knot))

Number of nodes to be loaded: 96
Next knot name: 51
Number of nodes to be loaded: 96
Next knot name: 52
Number of nodes to be loaded: 96
Next knot name: 53
Number of nodes to be loaded: 96
Next knot name: 54
Number of nodes to be loaded: 96
Next knot name: 55
Number of nodes to be loaded: 96
Next knot name: 56
Number of nodes to be loaded: 96
Next knot name: 57
Number of nodes to be loaded: 96
Next knot name: 58
Number of nodes to be loaded: 64
Next knot name: 59
Number of nodes to be loaded: 48
Next knot name: 60
Number of nodes to be loaded: 96
Next knot name: 61
Number of nodes to be loaded: 96


In [ ]:
req = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'))
req_string = req.read()
req.close()

new_req = req_string[0:req_string.find('\n')-1]+'8'+ req_string[req_string.find('\n'):]
req_overwrite = open(op.join(op.split(my_image.docker_path)[0], 'requirements.txt'), 'w')
req_overwrite.write(new_req)
req_overwrite.close()

In [ ]:
# github_installs=('https://github.com/ccurtis7/diff_classifier.git@scaleup')
# my_image = ck.DockerImage(func=kn.download_and_track, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

# docker_file = open(my_image.docker_path)
# docker_string = docker_file.read()
# docker_file.close()

# cut1 = docker_string.find('# Create a default user')
# cut2 = docker_string[cut1:].find('\n\n') + cut1
# new_docker = docker_string[0:cut1]+docker_string[cut2:]
# docker_overwrite = open(my_image.docker_path, 'w')
# docker_overwrite.write(new_docker)
# docker_overwrite.close()

# knot1 = ck.Knot(name='download_and_track_with_my_image_27',
#                docker_image = my_image,
#                pars_policies=('AmazonS3FullAccess',))
# result_futures = knot1.map(to_track)

In [ ]:
knot1.view_jobs()

In [ ]:
knot1.clobber()

In [ ]:
ck.Knot(name='download_and_track_with_my_image_30').clobber()

In [ ]:

to_clob = [12, 2, 22, 1, 3, 31, 32, 33, 34, 35, 36, 37, 4, 5]
for num in to_clob:
    ck.Knot(name='download_and_track_with_my_image_{}'.format(num))


In [ ]:
ck.Knot(name='download_and_track_with_my_image_31').clobber()

In [ ]:
for key in knot:
    print key
    knot[key].clobber()

In [ ]:
for key in knot